In [1]:
import shutil
import pandas as pd
import numpy as np
import os
import transformers
from transformers import  AutoModelForSequenceClassification, DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
from datasets import load_dataset
from sklearn.model_selection import train_test_split 

# the name of the pre-trained model we want to use
MODEL_NAME = "bert-base-uncased" 

c:\Users\Wenhao\miniconda3\envs\deep_wa_bert\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data Loading and Preprocessing

In [2]:
data = pd.read_parquet("../labeled_post_enriched_bert.parquet")
data = data.drop(["index", "link_flair_text_y", "Unnamed: 0"], axis=1)
data = data.rename({"link_flair_text_x": "link_flair_text"}, axis=1)

In [3]:
data.columns

Index(['link_flair_text', 'num_comments', 'over_18', 'score', 'url',
       'selftext', 'title', 'id', 'edited', 'is_self', 'permalink', 'downs',
       'ups', 'created', 'date', 'Topic', 'Count', 'Name', 'Representation',
       'verdict', 'is_asshole', 'demonyms', 'demonyms_words_count',
       'demonyms_unique_count', 'features', 'title_uppercase_count',
       'title_word_count', 'title_profanity_count', 'avg_word_length',
       'stop_word_count', 'numerics_count', 'uppercase_words_count',
       'sentence_count', 'avg_sentence_length', 'profanity_count', 'gender',
       'age', 'llama_gender', 'llama_age', 'gender_F', 'gender_M', 'gender_U',
       'title_anger', 'title_disgust', 'title_fear', 'title_joy',
       'title_others', 'title_sadness', 'title_surprise', 'selftext_anger',
       'selftext_disgust', 'selftext_fear', 'selftext_joy', 'selftext_others',
       'selftext_sadness', 'selftext_surprise'],
      dtype='object')

In [4]:
# data = pd.read_parquet("../labeled_post.parquet")
# data.shape

### Creating the label

In [5]:
data["link_flair_text"].value_counts()

Not the A-hole     55095
Asshole            13142
No A-holes here     3470
Everyone Sucks      2951
Name: link_flair_text, dtype: int64

In [6]:
flairs = {
    "Not the A-hole": "NTA",
    "No A-holes here": "NTA",
    "Asshole": "YTA",
    "Everyone Sucks": "YTA"
}

# filter data and label verdicts
labeled_data = data.loc[data.link_flair_text.isin(flairs)].copy()
labeled_data["verdict"] = labeled_data.link_flair_text.map(flairs)
labeled_data["verdict"] = (labeled_data["verdict"] == "YTA").astype(int)
labeled_data["verdict"].value_counts()

0    58565
1    16093
Name: verdict, dtype: int64

In [7]:
negative_data = labeled_data[labeled_data["verdict"] == 0]
positive_data = labeled_data[labeled_data["verdict"] == 1]
negative_data = negative_data.sample(len(positive_data), random_state=547)

In [8]:
labeled_data = pd.concat([negative_data, positive_data])
labeled_data["verdict"].value_counts()

0    16093
1    16093
Name: verdict, dtype: int64

### Creating the textual input

In [9]:
labeled_data["gender"] = labeled_data["gender"].map({"F": "Female", "M": "Male", "U": "Unknown"})
labeled_data["age"] = labeled_data["age"].fillna(-1).astype(int).astype(str)

In [10]:
sentiment_cols = ['title_anger', 'title_disgust', 'title_fear', 'title_joy',
       'title_others', 'title_sadness', 'title_surprise', 'selftext_anger',
       'selftext_disgust', 'selftext_fear', 'selftext_joy', 'selftext_others',
       'selftext_sadness', 'selftext_surprise']
for col in sentiment_cols:
    labeled_data[col] = labeled_data[col].round(4).astype(str)

In [11]:
extra_features = ["gender", "age", "Name", 'title_anger', 'title_disgust', 'title_fear', 'title_joy',
       'title_others', 'title_sadness', 'title_surprise', 'selftext_anger',
       'selftext_disgust', 'selftext_fear', 'selftext_joy', 'selftext_others',
       'selftext_sadness', 'selftext_surprise']

In [12]:
labeled_data["input"] = "title: " + labeled_data["title"] + " / text: " + labeled_data["selftext"]

In [12]:
# labeled_data["input"] = "gender: " + labeled_data["gender"] + " / age: " + labeled_data["age"] + " / topic: " + labeled_data["Name"] + \
#     " / text anger score: " + labeled_data["selftext_anger"] +  " / text disgust score: " + labeled_data["selftext_disgust"] + \
#     " / text fear score: " + labeled_data["selftext_fear"] +  " / text joy score: " + labeled_data["selftext_joy"] + \
#     " / text others score: " + labeled_data["selftext_others"] +  " / text sadness score: " + labeled_data["selftext_sadness"] + \
#     " / text surprise score: " + labeled_data["selftext_surprise"] + \
#     " / title: " + labeled_data["title"] + " / text: " + labeled_data["selftext"]
# labeled_data["input"] = "title: " + labeled_data["title"] + " / text: " + labeled_data["selftext"]

In [13]:
# labeled_data["input"] = "gender: " + labeled_data["gender"] + " / age: " + labeled_data["age"] + " / topic: " + labeled_data["Name"] + \
#     " / title anger score: " + labeled_data["title_anger"] +  " / title disgust score: " + labeled_data["title_disgust"] + \
#     " / title fear score: " + labeled_data["title_fear"] +  " / title joy score: " + labeled_data["title_joy"] + \
#     " / title others score: " + labeled_data["title_others"] +  " / title sadness score: " + labeled_data["title_sadness"] + \
#     " / title surprise score: " + labeled_data["title_surprise"] +  \
#     " / text anger score: " + labeled_data["selftext_anger"] +  " / text disgust score: " + labeled_data["selftext_disgust"] + \
#     " / text fear score: " + labeled_data["selftext_fear"] +  " / text joy score: " + labeled_data["selftext_joy"] + \
#     " / text others score: " + labeled_data["selftext_others"] +  " / text sadness score: " + labeled_data["selftext_sadness"] + \
#     " / text surprise score: " + labeled_data["selftext_surprise"] + \
#     " / title: " + labeled_data["title"] + " / text: " + labeled_data["selftext"]

In [13]:
labeled_data["input"].iloc[1]

'title: AITA for doubting my BF’s plan to start a small business? / text: For context I (27F) am dating my bf (32m) for about 2 years now. We have started to have some larger discussions regarding family planning, finances, family, marriage, etc. you get the gist. \n\nMy BF and I have very different backgrounds, we both work in corporate and have great benefits, I am child free, have absolutely no debt, trying to buy my first house, and am doing well for myself generally speaking. My BF has a son from a previous relationship, significant debt, family issues, living month to month, etc. it’s a lot. \n\nThrough discussing everything he has mentioned in about 6-10 years he’d want to start a brick and mortar athletic wear store. I’ve gone to business school (big flex I know lol), so I immediately asked questions about it not being profitable for several years, no PTO, working 10- 12 hour days, weekends, having a very sizable business loan, retirement, living off only my income for a long t

In [14]:
from gensim.parsing import remove_stopwords, strip_numeric, strip_punctuation, strip_multiple_whitespaces

# labeled_data["title_text"] = labeled_data["title"] + " --- " + labeled_data["selftext"]
labeled_data["input"] = labeled_data["input"].str.lower()
labeled_data["input"] = labeled_data["input"].apply(strip_multiple_whitespaces)
labeled_data = labeled_data.replace(to_replace=[''], value=np.nan).dropna(subset=["input", "verdict"])

In [15]:
final_data = labeled_data[["input", "verdict"]]
final_data.head()

,input,verdict
8157,title: aita for not wanted babysit my niece fo...,0
59194,title: aita for doubting my bf’s plan to start...,0
45681,title: aita for not telling my mom i was writi...,0
10748,title: wibta for going to prom with a boy? / t...,0
35567,title: aita for hiding that i'm in a relations...,0


### Data splitting

In [16]:
train, val = train_test_split(final_data, test_size=0.2, random_state=547)
train.to_csv("./data/train.csv", index=False)
val.to_csv("./data/val.csv", index=False)

In [17]:
dataset = load_dataset("csv", data_files={"train": "./data/train.csv", "val": "./data/val.csv"})

Using custom data configuration default-ece31272f485a1e4


Extracting data files: 100%|██████████| 2/2 [00:00<00:00, 1003.78it/s]


Dataset csv downloaded and prepared to C:\Users\Wenhao\.cache\huggingface\datasets\csv\default-ece31272f485a1e4\0.0.0\652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


100%|██████████| 2/2 [00:00<00:00, 308.06it/s]


## Tokenization

In [18]:
from transformers import AutoTokenizer

### Preprocess Setup ###
# tokenization hyperparameters
PADDING = 'max_length' # padding strategy
PADDING_SIDE = 'right' # the side on which the model should have padding applied
TRUNCATION = True # truncate strategy
TRUNCATION_SIDE = 'right' # the side on which the model should have truncation applied
MAX_LEN = 512 # maximum length to use by one of the truncation/padding parameters

# Load the pre-trained tokenmizer ###
TOKENIZER = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    padding_side=PADDING_SIDE,
    truncation_side=TRUNCATION_SIDE,
)

# Define the preprocess function ###
def preprocess_function(examples):
    """
    Preprocess the description field
    ---
    Arguments:
    examples (str, List[str], List[List[str]]: the sequence or batch of sequences to be encoded/tokenized

    Returns:
    tokenized (transformers.BatchEncoding): tokenized descriptions 
    """
    tokenized = TOKENIZER(
        examples["input"],
        padding=PADDING,
        truncation=TRUNCATION,
        max_length=MAX_LEN,
    )

    return tokenized

In [19]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns("input")

Parameter 'function'=<function preprocess_function at 0x000001D790881630> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
100%|██████████| 7/7 [00:01<00:00,  3.98ba/s]


In [20]:
tokenized_dataset = tokenized_dataset.rename_column("verdict", "label")

## Evaluation

In [21]:
import numpy as np
from datasets import load_metric
from sklearn.metrics import matthews_corrcoef, cohen_kappa_score, confusion_matrix, balanced_accuracy_score

### Evaluation Metrics ###
metric_acc = load_metric("accuracy")
metric_f1 = load_metric("f1")
metric_precision = load_metric("precision")
metric_recall = load_metric("recall")
metric_auc = load_metric("roc_auc")

def compute_metrics(eval_pred):
    """
    Compute the metrics 
    ---
    Arguments:
    eval_pred (tuple): the predicted logits and truth labels

    Returns:
    metrics (dict{str: float}): contains the computed metrics 
    """
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    prediction_scores = np.max(logits, axis=-1)
    print(logits.shape, labels.shape)
    print(predictions.shape, prediction_scores.shape)

    pred_true = np.count_nonzero(predictions)
    pred_false = predictions.shape[0] - pred_true
    actual_true = np.count_nonzero(labels)
    actual_false = labels.shape[0] - actual_true

    acc = metric_acc.compute(predictions=predictions, references=labels)['accuracy']
    f1 = metric_f1.compute(predictions=predictions, references=labels)['f1']
    precision = metric_precision.compute(predictions=predictions, references=labels)['precision']
    recall = metric_recall.compute(predictions=predictions, references=labels)['recall']
    roc_auc = metric_auc.compute(prediction_scores=predictions, references=labels)['roc_auc']
    matthews_correlation = matthews_corrcoef(y_true=labels, y_pred=predictions)
    cohen_kappa = cohen_kappa_score(y1=labels, y2=predictions)
    balanced_accuracy = balanced_accuracy_score(y_true=labels, y_pred=predictions)

    tn, fp, fn, tp = confusion_matrix(y_true=labels, y_pred=predictions).ravel()
    specificity = tn / (tn + fp)
    sensitivity = tp / (tp + fn)
    informedness = specificity + sensitivity - 1

    metrics = {
        "pred_true": pred_true,
        "pred_false": pred_false,
        "actual_true": actual_true,
        "actual_false": actual_false,
        "accuracy": acc,
        "f1_score": f1,
        "precision": precision,
        "recall": recall,
        "roc_auc": roc_auc,
        "matthews_correlation": matthews_correlation,
        "cohen_kappa": cohen_kappa,
        "true_negative": tn,
        "false_positive": fp,
        "false_negative": fn,
        "true_positive": tp,
        "specificity": specificity,
        "sensitivity": sensitivity,
        "informedness": informedness,
        "balanced_accuracy": balanced_accuracy
    }
    return metrics

## Model fitting

### Hyperparameters

In [22]:
# model hyperparameters
CLASSIFIER_DROPOUT = 0.15 # dropout ratio for the classification head
NUM_CLASSES = 2 # number of classes

# optimization hyperparameters ###
SEED = 547 # random seed for splitting the data into batches
BATCH_SIZE = 16 # batch size for both training and evaluation
GRAD_ACC_STEPS = 4 # number of steps for gradient accumulation
LR = 5e-5 # initial learning rate
WEIGHT_DECAY = 2e-3 # weight decay to apply in the AdamW optimizer
EPOCHS = 5 # total number of training epochs 
LR_SCHEDULER = "cosine" # type of learning rate scheduler
STRATEGY = "steps" # strategy for logging, evaluation, and saving
STEPS = 200 # number of steps for logging, evaluation, and saving
EVAL_METRIC = "f1_score" # metric for selecting the best model

### Fitting the model

In [23]:
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    classifier_dropout=CLASSIFIER_DROPOUT,
    num_labels=NUM_CLASSES,
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [24]:
# set up the training arguments
training_args = TrainingArguments(
    output_dir="./model",
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACC_STEPS,
    learning_rate=LR,
    weight_decay=WEIGHT_DECAY, 
    num_train_epochs=EPOCHS,
    lr_scheduler_type=LR_SCHEDULER,
    evaluation_strategy=STRATEGY,
    logging_strategy=STRATEGY, 
    save_strategy=STRATEGY,
    eval_steps=STEPS,
    logging_steps=STEPS,
    save_steps=STEPS,
    seed=SEED,
    load_best_model_at_end=True,
    metric_for_best_model=EVAL_METRIC,
    report_to="none"
)

In [25]:
# set up the trainer 
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['val'],
    tokenizer=TOKENIZER,   
    compute_metrics=compute_metrics,
)

In [26]:
try:
    shutil.rmtree("./model") # remove possible cache
    # shutil.rmtree(best_model_dir)
    # os.remove(best_model_dir_zip)
    os.mkdir("./model")
except:
    None

In [27]:
trainer.train()

c:\Users\Wenhao\miniconda3\envs\deep_wa_bert\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 25748
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 4
  Total optimization steps = 2010
 10%|▉         | 200/2010 [04:26<40:43,  1.35s/it]***** Running Evaluation *****
  Num examples = 6438
  Batch size = 16


{'loss': 0.6892, 'learning_rate': 4.8788458556153635e-05, 'epoch': 0.5}


                                                  
 10%|▉         | 200/2010 [05:16<40:43,  1.35s/it]Saving model checkpoint to ./model\checkpoint-200
Configuration saved in ./model\checkpoint-200\config.json


(6438, 2) (6438,)
(6438,) (6438,)
{'eval_loss': 0.6760535836219788, 'eval_pred_true': 2841, 'eval_pred_false': 3597, 'eval_actual_true': 3177, 'eval_actual_false': 3261, 'eval_accuracy': 0.5986331158744952, 'eval_f1_score': 0.5706214689265536, 'eval_precision': 0.6043646603308694, 'eval_recall': 0.5404469625432798, 'eval_roc_auc': 0.5978837081958962, 'eval_matthews_correlation': 0.1971145034740857, 'eval_cohen_kappa': 0.19603444201357323, 'eval_true_negative': 2137, 'eval_false_positive': 1124, 'eval_false_negative': 1460, 'eval_true_positive': 1717, 'eval_specificity': 0.6553204538485127, 'eval_sensitivity': 0.5404469625432798, 'eval_informedness': 0.19576741639179263, 'eval_balanced_accuracy': 0.5978837081958963, 'eval_runtime': 49.5749, 'eval_samples_per_second': 129.864, 'eval_steps_per_second': 8.129, 'epoch': 0.5}


Model weights saved in ./model\checkpoint-200\pytorch_model.bin
tokenizer config file saved in ./model\checkpoint-200\tokenizer_config.json
Special tokens file saved in ./model\checkpoint-200\special_tokens_map.json
 20%|█▉        | 400/2010 [09:54<37:46,  1.41s/it]  ***** Running Evaluation *****
  Num examples = 6438
  Batch size = 16


...

                                                   
100%|█████████▉| 2000/2010 [57:34<00:14,  1.48s/it]Saving model checkpoint to ./model\checkpoint-2000
Configuration saved in ./model\checkpoint-2000\config.json


(6438, 2) (6438,)
(6438,) (6438,)
{'eval_loss': 1.1419950723648071, 'eval_pred_true': 3251, 'eval_pred_false': 3187, 'eval_actual_true': 3177, 'eval_actual_false': 3261, 'eval_accuracy': 0.614165890027959, 'eval_f1_score': 0.6135656502800247, 'eval_precision': 0.6065825899723162, 'eval_recall': 0.6207113629209946, 'eval_roc_auc': 0.6142501923467285, 'eval_matthews_correlation': 0.22849222455774845, 'eval_cohen_kappa': 0.22843185643962038, 'eval_true_negative': 1982, 'eval_false_positive': 1279, 'eval_false_negative': 1205, 'eval_true_positive': 1972, 'eval_specificity': 0.6077890217724624, 'eval_sensitivity': 0.6207113629209946, 'eval_informedness': 0.22850038469345701, 'eval_balanced_accuracy': 0.6142501923467285, 'eval_runtime': 57.1258, 'eval_samples_per_second': 112.699, 'eval_steps_per_second': 7.055, 'epoch': 4.97}


Model weights saved in ./model\checkpoint-2000\pytorch_model.bin
tokenizer config file saved in ./model\checkpoint-2000\tokenizer_config.json
Special tokens file saved in ./model\checkpoint-2000\special_tokens_map.json
100%|██████████| 2010/2010 [57:50<00:00,  2.18s/it]

Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./model\checkpoint-400 (score: 0.6690918213748119).
100%|██████████| 2010/2010 [57:51<00:00,  1.73s/it]

{'train_runtime': 3471.1018, 'train_samples_per_second': 37.089, 'train_steps_per_second': 0.579, 'train_loss': 0.46642771633110236, 'epoch': 5.0}


TrainOutput(global_step=2010, training_loss=0.46642771633110236, metrics={'train_runtime': 3471.1018, 'train_samples_per_second': 37.089, 'train_steps_per_second': 0.579, 'train_loss': 0.46642771633110236, 'epoch': 5.0})

In [28]:
model.save_pretrained("./basic_feature_model")

Configuration saved in ./basic_feature_model\config.json
Model weights saved in ./basic_feature_model\pytorch_model.bin


In [29]:
val_eval = {}

val_predictions = trainer.predict(tokenized_dataset["val"])
val_eval[0] = val_predictions.metrics

val_eval_df = pd.DataFrame.from_dict(val_eval).transpose()
val_eval_df.to_csv(f"metrics/basic_feature_val_evaluation.csv")

***** Running Prediction *****
  Num examples = 6438
  Batch size = 16
100%|██████████| 403/403 [00:48<00:00,  8.29it/s]

(6438, 2) (6438,)
(6438,) (6438,)


## Hyperparameter tuning

Here we tune three hyperparameters -- learning rate, weight_decay, dropout.

In [23]:
import itertools as it

# model hyperparameters
CLASSIFIER_DROPOUT = 0.15 # dropout ratio for the classification head
NUM_CLASSES = 2 # number of classes

# optimization hyperparameters ###
SEED = 547 # random seed for splitting the data into batches
BATCH_SIZE = 16 # batch size for both training and evaluation
GRAD_ACC_STEPS = 2 # number of steps for gradient accumulation
LR = 5e-5 # initial learning rate
WEIGHT_DECAY = 2e-3 # weight decay to apply in the AdamW optimizer
EPOCHS = 3 # total number of training epochs 
LR_SCHEDULER = "cosine" # type of learning rate scheduler
STRATEGY = "steps" # strategy for logging, evaluation, and saving
STEPS = 100 # number of steps for logging, evaluation, and saving
EVAL_METRIC = "f1_score" # metric for selecting the best model

HYPERPARAMETER_GRID = {
    "learning_rate": [5e-5, 3e-5, 2e-5], # recommended by the BERT authors
    "weight_decay": [1/8 * 1e-3, 1/4 * 1e-3, 1/2 * 1e-3], # recommended by the AdamW authors
    "dropout": [1e-1, 3e-1, 5e-1]
}

ALL_COMBINATIONS = it.product(*(HYPERPARAMETER_GRID[key] for key in HYPERPARAMETER_GRID))
ALL_COMBINATIONS = list(ALL_COMBINATIONS)

In [26]:
ALL_COMBINATIONS = ALL_COMBINATIONS[1:]

In [28]:
val_eval = {}

for combination in ALL_COMBINATIONS:
    try:
        shutil.rmtree("./model") # remove possible cache
        # shutil.rmtree(best_model_dir)
        # os.remove(best_model_dir_zip)
        os.mkdir("./model")
    except:
        None

    LR = combination[0]
    WEIGHT_DECAY = combination[1]
    CLASSIFIER_DROPOUT = combination[2]

    print(f"=== Learning rate = {LR}, Weight decay = {WEIGHT_DECAY}, Classifier dropout = {CLASSIFIER_DROPOUT} ===")

    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,
        classifier_dropout=CLASSIFIER_DROPOUT,
        num_labels=NUM_CLASSES,
    )

    # set up the training arguments
    training_args = TrainingArguments(
        output_dir="./model",
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        gradient_accumulation_steps=GRAD_ACC_STEPS,
        learning_rate=LR,
        weight_decay=WEIGHT_DECAY, 
        num_train_epochs=EPOCHS,
        lr_scheduler_type=LR_SCHEDULER,
        evaluation_strategy=STRATEGY,
        logging_strategy=STRATEGY, 
        save_strategy=STRATEGY,
        eval_steps=STEPS,
        logging_steps=STEPS,
        save_steps=STEPS,
        seed=SEED,
        load_best_model_at_end=True,
        metric_for_best_model=EVAL_METRIC,
        report_to="none"
    )

    # set up the trainer 
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset['train'],
        eval_dataset=tokenized_dataset['val'],
        tokenizer=TOKENIZER,   
        compute_metrics=compute_metrics,
    )

    trainer.train()

    val_predictions = trainer.predict(tokenized_dataset["val"])
    val_eval[combination] = val_predictions.metrics

    val_eval_df = pd.DataFrame.from_dict(val_eval).transpose()
    val_eval_df.to_csv(f"metrics/val_evaluation_2.csv")

=== Learning rate = 5e-05, Weight decay = 0.000125, Classifier dropout = 0.3 ===


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

{'loss': 0.7004, 'learning_rate': 4.97887664223157e-05, 'epoch': 0.12}



  4%|▍         | 100/2415 [02:08<28:02,  1.38it/s]Saving model checkpoint to ./model\checkpoint-100
Configuration saved in ./model\checkpoint-100\config.json


(6438, 2) (6438,)
(6438,) (6438,)
{'eval_loss': 0.691122829914093, 'eval_pred_true': 21, 'eval_pred_false': 6417, 'eval_actual_true': 3177, 'eval_actual_false': 3261, 'eval_accuracy': 0.5079217148182665, 'eval_f1_score': 0.009380863039399624, 'eval_precision': 0.7142857142857143, 'eval_recall': 0.004721435316336166, 'eval_roc_auc': 0.5014407544566961, 'eval_matthews_correlation': 0.025265523843754474, 'eval_cohen_kappa': 0.0029188537537234716, 'eval_true_negative': 3255, 'eval_false_positive': 6, 'eval_false_negative': 3162, 'eval_true_positive': 15, 'eval_specificity': 0.9981600735970562, 'eval_sensitivity': 0.004721435316336166, 'eval_informedness': 0.002881508913392228, 'eval_balanced_accuracy': 0.5014407544566961, 'eval_runtime': 53.2701, 'eval_samples_per_second': 120.856, 'eval_steps_per_second': 7.565, 'epoch': 0.12}


...

{'loss': 0.689, 'learning_rate': 3.764055752534714e-05, 'epoch': 0.99}



 33%|███▎      | 800/2415 [19:24<22:02,  1.22it/s]Saving model checkpoint to ./model\checkpoint-800
Configuration saved in ./model\checkpoint-800\config.json


(6438, 2) (6438,)
(6438,) (6438,)
{'eval_loss': 0.6843065023422241, 'eval_pred_true': 4384, 'eval_pred_false': 2054, 'eval_actual_true': 3177, 'eval_actual_false': 3261, 'eval_accuracy': 0.5695868282075178, 'eval_f1_score': 0.633514085438434, 'eval_precision': 0.5463047445255474, 'eval_recall': 0.7538558388416745, 'eval_roc_auc': 0.5719601181328888, 'eval_matthews_correlation': 0.1543726565090383, 'eval_cohen_kappa': 0.14321944202470271, 'eval_true_negative': 1272, 'eval_false_positive': 1989, 'eval_false_negative': 782, 'eval_true_positive': 2395, 'eval_specificity': 0.39006439742410304, 'eval_sensitivity': 0.7538558388416745, 'eval_informedness': 0.14392023626577766, 'eval_balanced_accuracy': 0.5719601181328888, 'eval_runtime': 62.0285, 'eval_samples_per_second': 103.791, 'eval_steps_per_second': 6.497, 'epoch': 0.99}


Model weights saved in ./model\checkpoint-800\pytorch_model.bin
tokenizer config file saved in ./model\checkpoint-800\tokenizer_config.json
Special tokens file saved in ./model\checkpoint-800\special_tokens_map.json
 35%|███▍      | 841/2415 [19:59<22:22,  1.17it/s]  

KeyboardInterrupt: 